In [1]:
import starepandas
import sqlalchemy
import geopandas
import pystare
import matplotlib.pyplot as plt
import glob
import pickle
import pandas

In [2]:
db_path = 'catalogs/catalog.sqlite' 
uri = 'sqlite:///{db_path}'.format(db_path=db_path)
engine = sqlalchemy.create_engine(uri)

# Making Catalogs

## CLDMSK NOAA20

In [3]:
path = '/tablespace/caribbean/CLDMSK_L2_VIIRS_NOAA20/'
granule_trunk = 'CLDMSK_L2_VIIRS'

catalog_cldmsk_noaa20 = starepandas.folder2catalog(path=path, 
                                                       granule_trunk=granule_trunk,
                                                       granule_extension='nc')

catalog_cldmsk_noaa20.to_sql(name='cldmsk_noaa_20', con=engine, if_exists='replace', index=False)
catalog_cldmsk_noaa20.to_pickle('catalogs/catalog_cldmsk_noaa20.pickle')

## CLDMSK SNPP

In [7]:
path = '/tablespace/caribbean/CLDMSK_L2_VIIRS_SNPP/'

catalog_cldmsk_snpp = starepandas.folder2catalog(path=path, 
                                                 granule_trunk=granule_trunk,
                                                 granule_extension='nc')

catalog_cldmsk_snpp.to_sql(name='catalog_cldmsk_snpp', con=engine, if_exists='replace', index=False)
catalog_cldmsk_snpp.to_pickle('catalogs/catalog_cldmsk_snpp.pickle')

## VJ103DNB

In [8]:
path = '/tablespace/caribbean/VJ103DNB/'
granule_trunk = 'VJ103DNB'

catalog_vj103 = starepandas.folder2catalog(path=path, 
                                               granule_trunk=granule_trunk,
                                               granule_extension='nc')

catalog_vj103.to_sql(name='vj103', con=engine, if_exists='replace', index=False)
catalog_vj103.to_pickle('catalogs/catalog_vj103.pickle')

## VNP03DNB

In [9]:
path = '/tablespace/caribbean/VNP03DNB/'
granule_trunk = 'VNP03DNB'

catalog_vnp03 = starepandas.folder2catalog(path=path, 
                                           granule_trunk=granule_trunk,
                                           granule_extension='nc')

catalog_vnp03.to_sql(name='vj103', con=engine, if_exists='replace', index=False)
catalog_vnp03.to_pickle('catalogs/catalog_vnp03.pickle')

## VJ102DNB 

In [11]:
path = '/tablespace/caribbean/VJ102DNB/'
granule_trunk = 'VJ102DNB'
granule_extension = 'nc'

term = '{path}/{granule_trunk}*[!_stare].{ext}'
term = term.format(path=path, granule_trunk=granule_trunk, ext=granule_extension)
granule_paths = glob.glob(term)

catalog_vj102 = pandas.DataFrame()
for granule_path in granule_paths:    
    vnp02 = starepandas.io.file.VNP02DNB(vnp02_path=granule_path,
                                         vnp03_folder='/tablespace/caribbean/VJ103DNB/')
    vnp02.read_sidecar_cover()
    vnp02.read_timestamps()    
    
    row = {}
    row['granule_path'] = granule_path
    row['companion_path'] = vnp02.vnp03.file_path
    row['sidecar_path'] = vnp02.sidecar_path
    row['stare_cover'] = vnp02.stare_cover
    row['begining'] = vnp02.ts_start
    row['ending'] = vnp02.ts_end
    
    catalog_vj102 = catalog_vj102.append(row, ignore_index=True)

catalog_vj102 = starepandas.STAREDataFrame(catalog_vj102, stare='stare_cover')

catalog_vj102.to_sql(name='vj102', con=engine, if_exists='replace', index=False)
catalog_vj102.to_pickle('catalogs/catalog_vj102.pickle')

## VNP02DNB

In [ ]:
path = '/tablespace/caribbean/VNP02DNB/'
granule_trunk = 'VNP02DNB'
granule_extension = 'nc'

term = '{path}/{granule_trunk}*[!_stare].{ext}'
term = term.format(path=path, granule_trunk=granule_trunk, ext=granule_extension)
granule_paths = glob.glob(term)

catalog_vnp02 = pandas.DataFrame()
for granule_path in granule_paths:    
    vnp02 = starepandas.io.file.VNP02DNB(vnp02_path=granule_path,
                                         vnp03_folder='/tablespace/caribbean/VNP03DNB/')
    vnp02.read_sidecar_cover()
    vnp02.read_timestamps()    
    
    row = {}
    row['granule_path'] = granule_path
    row['companion_path'] = vnp02.vnp03.file_path
    row['sidecar_path'] = vnp02.sidecar_path
    row['stare_cover'] = vnp02.stare_cover
    row['begining'] = vnp02.ts_start
    row['ending'] = vnp02.ts_end
    
    catalog_vnp02 = catalog_vnp02.append(row, ignore_index=True)

catalog_vnp02 = starepandas.STAREDataFrame(catalog_vnp02, stare='stare_cover')

catalog_vnp02.to_sql(name='vnp02', con=engine, if_exists='replace', index=False)
catalog_vnp02.to_pickle('catalogs/catalog_vnp02.pickle')

## Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=200)
ax.grid(True)

catalog_vj102[0:5].set_trixels(inplace=False).plot(ax=ax, trixels=True, column='granule_path')